# 🔲 ICHTB Tensor Box Visualization

> **Inverse Cartesian + Heisenberg Tensor Box**
>
> Interactive 3D visualization of the 6-fan recursive coordinate system

---

## Overview

This notebook renders the **ICHTB (Intent-Tensor Collapse Handler + Tensor Box)** — a recursive coordinate system where:

- **6 pyramidal fans** (Δ₁-Δ₆) converge inward to a central anchor
- **i₀ ∈ ℂ** — the imaginary scalar root at center
- Each fan represents a **collapse operator gate**

### Fan-Operator Mapping:

| Fan | Axis | Operator | Color |
|-----|------|----------|-------|
| Δ₁ | +Y | ∇Φ | Dark Green |
| Δ₂ | −Y | ∇×F | Light Green |
| Δ₃ | +X | +∇²Φ | Dark Red |
| Δ₄ | −X | −∇²Φ | Light Red |
| Δ₅ | +Z | ∂Φ/∂t | Dark Blue |
| Δ₆ | −Z | Φ=i₀ | Light Blue |

---

## Cell 1: Imports and Setup

In [ ]:
# Core libraries
import numpy as np
import plotly.graph_objects as go
from itertools import product

print("✓ Libraries loaded")
print(f"  NumPy: {np.__version__}")

---

## Cell 2: Configuration Parameters

In [ ]:
# =============================================================================
# ICHTB CONFIGURATION
# =============================================================================

# Lattice parameters
LATTICE_SIZE = 4          # Half-width of cube (full cube is 2*SIZE+1)
SPACING = 1.0             # Distance between lattice points

# Visual parameters
NODE_SIZE = 3             # Size of lattice nodes
EDGE_WIDTH = 1            # Width of lattice edges
PYRAMID_OPACITY = 0.4     # Transparency of fan pyramids
ANCHOR_SIZE = 15          # Size of central i₀ marker

# Color scheme (Dark = positive axis, Light = negative axis)
COLORS = {
    'Δ₁_+Y': 'darkgreen',      # ∇Φ
    'Δ₂_-Y': 'lightgreen',     # ∇×F
    'Δ₃_+X': 'darkred',        # +∇²Φ
    'Δ₄_-X': 'lightcoral',     # -∇²Φ
    'Δ₅_+Z': 'darkblue',       # ∂Φ/∂t
    'Δ₆_-Z': 'lightblue',      # Φ=i₀
    'nodes': 'gray',
    'edges': 'lightgray',
    'anchor': 'gold'
}

print("✓ Configuration set")
print(f"  Lattice: {2*LATTICE_SIZE+1}³ = {(2*LATTICE_SIZE+1)**3} points")

---

## Cell 3: Lattice Generation

In [ ]:
def generate_lattice(size, spacing):
    """
    Generate a cubic lattice centered at origin.
    
    Parameters:
        size: Half-width (lattice spans -size to +size)
        spacing: Distance between adjacent points
        
    Returns:
        nodes: (N, 3) array of node positions
        edges: List of (start_idx, end_idx) pairs
    """
    # Generate grid coordinates
    coords = np.arange(-size, size + 1) * spacing
    
    # Create all node positions
    nodes = np.array(list(product(coords, repeat=3)))
    
    # Build edge list (connect adjacent nodes)
    edges = []
    n_per_dim = 2 * size + 1
    
    for i, (x, y, z) in enumerate(nodes):
        # Find index in flattened array
        ix = int((x / spacing) + size)
        iy = int((y / spacing) + size)
        iz = int((z / spacing) + size)
        
        # Connect to +x neighbor
        if ix < n_per_dim - 1:
            j = i + n_per_dim * n_per_dim
            if j < len(nodes):
                edges.append((i, i + 1))
        
        # Connect to +y neighbor  
        if iy < n_per_dim - 1:
            edges.append((i, i + n_per_dim))
            
        # Connect to +z neighbor
        if iz < n_per_dim - 1:
            edges.append((i, i + n_per_dim * n_per_dim))
    
    return nodes, edges

# Generate lattice
nodes, edges = generate_lattice(LATTICE_SIZE, SPACING)

print(f"✓ Lattice generated")
print(f"  Nodes: {len(nodes)}")
print(f"  Edges: {len(edges)}")

---

## Cell 4: Pyramid Geometry for Fans

In [ ]:
def create_pyramid_mesh(face_direction, size, apex=(0, 0, 0)):
    """
    Create a pyramid mesh for one fan of the ICHTB.
    
    The pyramid has:
    - Base on the outer face of the cube
    - Apex at the center (i₀)
    
    Parameters:
        face_direction: '+X', '-X', '+Y', '-Y', '+Z', '-Z'
        size: Half-width of cube
        apex: (x, y, z) of pyramid apex (center i₀)
        
    Returns:
        dict with 'x', 'y', 'z', 'i', 'j', 'k' for Mesh3d
    """
    s = size * SPACING
    ax, ay, az = apex
    
    # Define base corners based on face direction
    if face_direction == '+X':
        base = [(s, -s, -s), (s, s, -s), (s, s, s), (s, -s, s)]
    elif face_direction == '-X':
        base = [(-s, -s, -s), (-s, -s, s), (-s, s, s), (-s, s, -s)]
    elif face_direction == '+Y':
        base = [(-s, s, -s), (s, s, -s), (s, s, s), (-s, s, s)]
    elif face_direction == '-Y':
        base = [(-s, -s, -s), (-s, -s, s), (s, -s, s), (s, -s, -s)]
    elif face_direction == '+Z':
        base = [(-s, -s, s), (s, -s, s), (s, s, s), (-s, s, s)]
    elif face_direction == '-Z':
        base = [(-s, -s, -s), (-s, s, -s), (s, s, -s), (s, -s, -s)]
    else:
        raise ValueError(f"Unknown face: {face_direction}")
    
    # Vertices: 4 base corners + 1 apex
    vertices = base + [apex]
    x = [v[0] for v in vertices]
    y = [v[1] for v in vertices]
    z = [v[2] for v in vertices]
    
    # Triangular faces: 4 sides + 2 for base
    # Sides: each connects apex (index 4) to two adjacent base corners
    i = [0, 1, 2, 3, 0, 1]  # First vertex of each triangle
    j = [1, 2, 3, 0, 2, 3]  # Second vertex
    k = [4, 4, 4, 4, 2, 1]  # Third vertex (apex for sides, base for bottom)
    
    return {'x': x, 'y': y, 'z': z, 'i': i, 'j': j, 'k': k}

# Create all six fan pyramids
fans = {
    'Δ₁ (+Y): ∇Φ': ('+Y', COLORS['Δ₁_+Y']),
    'Δ₂ (−Y): ∇×F': ('-Y', COLORS['Δ₂_-Y']),
    'Δ₃ (+X): +∇²Φ': ('+X', COLORS['Δ₃_+X']),
    'Δ₄ (−X): −∇²Φ': ('-X', COLORS['Δ₄_-X']),
    'Δ₅ (+Z): ∂Φ/∂t': ('+Z', COLORS['Δ₅_+Z']),
    'Δ₆ (−Z): Φ=i₀': ('-Z', COLORS['Δ₆_-Z']),
}

print("✓ Pyramid geometry functions ready")
print(f"  6 fans defined with operator assignments")

---

## Cell 5: Build Plotly Traces

In [ ]:
def build_traces():
    """
    Build all Plotly traces for the ICHTB visualization.
    
    Returns:
        List of go.Scatter3d and go.Mesh3d traces
    """
    traces = []
    
    # --- Lattice Nodes ---
    traces.append(go.Scatter3d(
        x=nodes[:, 0],
        y=nodes[:, 1],
        z=nodes[:, 2],
        mode='markers',
        marker=dict(size=NODE_SIZE, color=COLORS['nodes'], opacity=0.5),
        name='Lattice Nodes',
        visible=True
    ))
    
    # --- Lattice Edges ---
    edge_x, edge_y, edge_z = [], [], []
    for start, end in edges:
        edge_x.extend([nodes[start, 0], nodes[end, 0], None])
        edge_y.extend([nodes[start, 1], nodes[end, 1], None])
        edge_z.extend([nodes[start, 2], nodes[end, 2], None])
    
    traces.append(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(width=EDGE_WIDTH, color=COLORS['edges']),
        name='Lattice Edges',
        visible=True
    ))
    
    # --- Central Anchor (i₀) ---
    traces.append(go.Scatter3d(
        x=[0], y=[0], z=[0],
        mode='markers+text',
        marker=dict(size=ANCHOR_SIZE, color=COLORS['anchor'], symbol='diamond'),
        text=['i₀'],
        textposition='top center',
        textfont=dict(size=14, color='black'),
        name='Anchor (i₀)',
        visible=True
    ))
    
    # --- Fan Pyramids ---
    for name, (direction, color) in fans.items():
        mesh = create_pyramid_mesh(direction, LATTICE_SIZE)
        traces.append(go.Mesh3d(
            x=mesh['x'], y=mesh['y'], z=mesh['z'],
            i=mesh['i'], j=mesh['j'], k=mesh['k'],
            color=color,
            opacity=PYRAMID_OPACITY,
            name=name,
            visible=True
        ))
    
    return traces

traces = build_traces()
print(f"✓ Built {len(traces)} traces")
print(f"  - 1 node scatter")
print(f"  - 1 edge scatter")
print(f"  - 1 anchor marker")
print(f"  - 6 fan meshes")

---

## Cell 6: Create Interactive Figure

In [ ]:
def create_figure(traces):
    """
    Create the complete Plotly figure with toggle buttons.
    """
    fig = go.Figure(data=traces)
    
    # Layout
    fig.update_layout(
        title=dict(
            text='<b>ICHTB Tensor Box</b><br><sup>Inverse Cartesian + Heisenberg Tensor Box</sup>',
            x=0.5,
            font=dict(size=20)
        ),
        scene=dict(
            xaxis_title='X (Curvature Axis)',
            yaxis_title='Y (Vector Axis)',
            zaxis_title='Z (Temporal Axis)',
            aspectmode='cube',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        legend=dict(
            yanchor='top',
            y=0.99,
            xanchor='left',
            x=0.01,
            bgcolor='rgba(255,255,255,0.8)'
        ),
        margin=dict(l=0, r=0, t=80, b=0),
        width=900,
        height=700
    )
    
    # Add toggle buttons for each fan
    buttons = []
    
    # "Show All" button
    buttons.append(dict(
        label='Show All',
        method='update',
        args=[{'visible': [True] * len(traces)}]
    ))
    
    # Individual fan toggles
    for i, name in enumerate(fans.keys()):
        # Create visibility list: always show nodes, edges, anchor
        vis = [True, True, True] + [False] * 6
        vis[3 + i] = True  # Show this fan
        buttons.append(dict(
            label=name.split(':')[0],  # Just "Δ₁ (+Y)"
            method='update',
            args=[{'visible': vis}]
        ))
    
    fig.update_layout(
        updatemenus=[dict(
            type='buttons',
            direction='down',
            buttons=buttons,
            pad=dict(r=10, t=10),
            showactive=True,
            x=1.0,
            xanchor='left',
            y=1.0,
            yanchor='top'
        )]
    )
    
    return fig

fig = create_figure(traces)
print("✓ Figure created with interactive controls")

---

## Cell 7: Display the ICHTB

In [ ]:
# Display the interactive visualization
fig.show()

---

## Interpretation Guide

### What You're Seeing:

1. **Lattice Grid**: The gray points and lines represent the shell grid structure within the ICHTB

2. **Central Anchor (i₀)**: The gold diamond at center is the **imaginary scalar root** — the zero-dimensional recursion seed

3. **Six Pyramidal Fans**: Each colored pyramid represents one **collapse operator gate**:
   - Pyramids point **inward** to i₀ (inverse topology)
   - Each face is a **permission evaluation surface**

### Fan Meanings:

| Fan | What It Does |
|-----|-------------|
| Δ₁ (Green) | Evaluates **gradient alignment** — is there collapse direction? |
| Δ₂ (Lt Green) | Evaluates **phase memory** — is there recursion echo? |
| Δ₃ (Red) | Evaluates **expansion** — is curvature opening? |
| Δ₄ (Lt Red) | Evaluates **compression** — is curvature closing? |
| Δ₅ (Blue) | Evaluates **emergence** — is collapse happening now? |
| Δ₆ (Lt Blue) | Evaluates **anchor resolution** — does it trace to i₀? |

### The Key Insight:

> **This is not a spatial coordinate system.**
>
> It is a **permission manifold** — a computational structure that determines
> where recursive collapse is allowed to form observable structure.

---

## References

- [Intent Tensor Theory: Coordinate System](https://intent-tensor-theory.com/coordinate-system/)
- [ICHTB Documentation](../docs/)
- [Collapse Operators](../mathematics/collapse_operators.md)